In [45]:
# Import necessary libraries:
import pandas as pd                       # for data manipulation and analysis
import numpy as np                        # for numerical operations (e.g., handling NaNs, arrays)
from datetime import datetime, timedelta  # for date and time manipulation

In [46]:
# Load the final dataset:
df = pd.read_csv('../data/cleaned_data/master_olist_dataset.csv')

In [47]:
# Define a list of column names that contain datetime information:
date_cols = [
    'order_purchase_timestamp',           # When the customer placed the order
    'order_approved_at',                  # When the order was approved
    'order_delivered_carrier_date',       # When the seller handed the item to the shipping carrier
    'order_delivered_customer_date',      # When the customer received the order
    'order_estimated_delivery_date',      # The estimated delivery date promised to the customer
    'shipping_limit_date',                # The deadline for the seller to ship the item
    'review_creation_date',               # When the customer created a review
    'review_answer_timestamp'             # When the review received a reply
]

# Loop through each column in the list:
for col in date_cols:
    # Convert the column to datetime format:
    # errors = 'coerce' will turn invalid or missing dates into NaT (Not a Time):
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [48]:
# Create an empty DataFrame to store temporal features:
temporal_feats = pd.DataFrame()

# Copy 'order_id' to use as a reference key for merging later:
temporal_feats['order_id'] = df['order_id']

# SLA-related features (measuring time differences between order lifecycle events):
# Time taken (in hours) from customer purchase to order approval:
temporal_feats['purchase_to_approve_hrs'] = (
    (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600
)

# Time taken (in hours) from approval to shipping handoff to carrier:
temporal_feats['approve_to_carrier_hrs'] = (
    (df['order_delivered_carrier_date'] - df['order_approved_at']).dt.total_seconds() / 3600
)

# Total time (in hours) from purchase to handoff to carrier:
temporal_feats['purchase_to_carrier_hrs'] = (
    (df['order_delivered_carrier_date'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600
)

# Time taken (in hours) for shipping from carrier to customer:
temporal_feats['carrier_to_customer_hrs'] = (
    (df['order_delivered_customer_date'] - df['order_delivered_carrier_date']).dt.total_seconds() / 3600
)

# Total delivery time (in hours) from order purchase to final delivery:
temporal_feats['purchase_to_customer_hrs'] = (
    (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600
)

# Estimated delivery window (in days) promised to customer at purchase time:
temporal_feats['est_delivery_window_days'] = (
    (df['order_estimated_delivery_date'] - df['order_purchase_timestamp']).dt.days
)

# Difference (in days) between actual delivery and the promised estimated date:
# (Positive = late, Negative = early, Zero = on-time):
temporal_feats['actual_delivery_vs_estimate_days'] = (
    (df['order_delivered_customer_date'] - df['order_estimated_delivery_date']).dt.days
)

# Boolean flag (true if delivery was late (actual delivery > estimate)):
temporal_feats['is_late'] = temporal_feats['actual_delivery_vs_estimate_days'] > 0

In [49]:
# Temporal purchase patterns (extract useful purchase time features):
# Hour of the day when the purchase occurred (0–23):
temporal_feats['purchase_hour'] = df['order_purchase_timestamp'].dt.hour

# Day of the week of purchase (0=Monday, 6=Sunday):
temporal_feats['purchase_dow'] = df['order_purchase_timestamp'].dt.dayofweek

# Month when the order was placed (1–12):
temporal_feats['purchase_month'] = df['order_purchase_timestamp'].dt.month

# Boolean flag (true if the purchase was on a weekend (Saturday or Sunday)):
temporal_feats['is_weekend'] = temporal_feats['purchase_dow'].isin([5, 6])

# Boolean flag (true if the order was placed in peak months (November or December)):
temporal_feats['is_peak_month'] = temporal_feats['purchase_month'].isin([11, 12])

In [50]:
# Brazil national holidays (month, day):
brazil_holidays = {
    (1, 1),   # New Year's Day
    (4, 21),  # Tiradentes Day
    (5, 1),   # Labour Day
    (9, 7),   # Independence Day
    (10, 12), # Our Lady of Aparecida
    (11, 2),  # All Souls' Day
    (11, 15), # Republic Day
    (11, 20), # Black Consciousness Day
    (12, 25), # Christmas Day
}

In [51]:
# Black Friday dates (4th Friday of November):
# Get all years from the dataset:
order_years = df['order_purchase_timestamp'].dropna().dt.year.unique()

# Compute the 4th Friday of November for each year:
black_fridays = set()
for year in order_years:
    # Start from November 1st of that year:
    nov_first = datetime(year, 11, 1)
    # Find first Friday:
    first_friday = nov_first + timedelta(days=(4 - nov_first.weekday()) % 7)
    # 4th Friday = first Friday + 21 days:
    fourth_friday = first_friday + timedelta(days=21)
    black_fridays.add((fourth_friday.month, fourth_friday.day))

# Add calculated Black Friday dates:
brazil_holidays.update(black_fridays)

In [52]:
# Extract (month, day) tuples from the purchase timestamp:
temporal_feats['purchase_month_day'] = list(zip(
    df['order_purchase_timestamp'].dt.month,
    df['order_purchase_timestamp'].dt.day
))

In [53]:
# Boolean flag if the (month, day) is in the holiday set:
temporal_feats['is_brazil_holiday'] = temporal_feats['purchase_month_day'].isin(brazil_holidays)

In [54]:
# Drop helper column:
temporal_feats.drop(columns='purchase_month_day', inplace=True)

In [55]:
# Show sample output:
print(temporal_feats.head())

# Check for missing values:
print("\nMissing values per column:")
print(temporal_feats.isnull().sum())

# Check distribution of lateness:
print("\nLate deliveries breakdown:")
print(temporal_feats['is_late'].value_counts())

# Purchase hour distribution:
print("\nPurchase hour value counts:")
print(temporal_feats['purchase_hour'].value_counts().sort_index())

# Brazil holiday purchases breakdown:
print("\nBrazil holiday purchases breakdown:")
print(temporal_feats['is_brazil_holiday'].value_counts())

                           order_id  purchase_to_approve_hrs  approve_to_carrier_hrs  purchase_to_carrier_hrs  carrier_to_customer_hrs  purchase_to_customer_hrs  est_delivery_window_days  actual_delivery_vs_estimate_days  is_late  purchase_hour  purchase_dow  purchase_month  is_weekend  is_peak_month  is_brazil_holiday
0  e481f51cbdc54678b7cc49136f2d6af7                 0.178333               56.795833                56.974167               145.503611                202.477778                        15                                -8    False             10             0              10       False          False              False
1  e481f51cbdc54678b7cc49136f2d6af7                 0.178333               56.795833                56.974167               145.503611                202.477778                        15                                -8    False             10             0              10       False          False              False
2  e481f51cbdc54678b7cc49136f2d6af7  

In [56]:
# Merge back:
df = df.merge(temporal_feats, on='order_id', how='left')

In [57]:
# Check:
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,has_review,product_category_name_english,purchase_to_approve_hrs,approve_to_carrier_hrs,purchase_to_carrier_hrs,carrier_to_customer_hrs,purchase_to_customer_hrs,est_delivery_window_days,actual_delivery_vs_estimate_days,is_late,purchase_hour,purchase_dow,purchase_month,is_weekend,is_peak_month,is_brazil_holiday
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,1.0,credit_card,1.0,18.12,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio correto e em boas condições. Apenas a caixa que veio bem amassada e danificada, o que ficará chato, pois se trata de um presente.",2017-10-11,2017-10-12 03:43:48,True,housewares,0.178333,56.795833,56.974167,145.503611,202.477778,15,-8,False,10,0,10,False,False,False
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,1.0,credit_card,1.0,18.12,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio correto e em boas condições. Apenas a caixa que veio bem amassada e danificada, o que ficará chato, pois se trata de um presente.",2017-10-11,2017-10-12 03:43:48,True,housewares,0.178333,56.795833,56.974167,145.503611,202.477778,15,-8,False,10,0,10,False,False,False
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,1.0,credit_card,1.0,18.12,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio correto e em boas condições. Apenas a caixa que veio bem amassada e danificada, o que ficará chato, pois se trata de um presente.",2017-10-11,2017-10-12 03:43:48,True,housewares,0.178333,56.795833,56.974167,145.503611,202.477778,15,-8,False,10,0,10,False,False,False
3,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,3.0,voucher,1.0,2.00,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio correto e em boas condições. Apenas a caixa que veio bem amassada e danificada, o que ficará chato, pois se trata de um presente.",2017-10-11,2017-10-12 03:43:48,True,housewares,0.1783

In [58]:
# Check:
df.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'seller_zip_code_prefix', 'seller_city', 'seller_state',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp', 'has_review',
       'product_category_name_english', 'purchase_to_approve_hrs',
     

In [59]:
# Create summary of all unique values for temporal features:
summary = []

for col in temporal_feats.columns:
    unique_vals = temporal_feats[col].dropna().unique()
    summary.append({
        'Column': col,
        'Unique Count': len(unique_vals),
        # Convert to list of strings to prevent truncation:
        'Unique Values': list(map(str, unique_vals))
    })

df_temporal_summary = pd.DataFrame(summary)

# Improve display settings in Jupyter/Notebook:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

df_temporal_summary

,Column,Unique Count,Unique Values
0,order_id,96266,"[e481f51cbdc54678b7cc49136f2d6af7, 53cdb2fc8bc7dce0b6741e2150273451, 47770eb9100c2d0c44946d9cf07ec65d, 949d5b44dbf5de918fe9c16f97b45f8a, ad21c59c0840e6cb83a9ceb5573f8159, a4591c265e18cb1dcee52889e2d8acc3, 6514b8ad8028c9f2cc2374ded245783f, 76c6e866289321a7c93b82b54852dc33, e69bfb5eb88e0ed6a785585b27e16dbf, e6ce16cb79ec1d90b1da9085a6118aeb, 34513ce0c4fab462a55830c0989c7edb, 82566a660a982b15fb86e904c8d32918, 5ff96c15d0b717ac6ad1f3d77225a350, 432aaf21d85167c2c86ec9448c4e42cc, dcb36b511fcac050b97cd5c05de84dc3, 403b97836b0c04a622354cf531062e5f, 116f0b09343b49556bbad5f35bee0cdf, 85ce859fd6dc634de8d2f1e290444043, 83018ec114eee8641c97e08f7b4e926f, 203096f03d82e0dffbc41ebc2e2bcfb7, f848643eec1d69395095eb3840d2051e, 2807d0e504d6d4894d41672727bc139f, 95266dbfb7e20354baba07964dac78d5, f3e7c359154d965827355f39d6b1fdac, fbf9ac61453ac646ce8ad9783d7d0af6, acce194856392f074dbf9dada14d8d82, dd78f560c270f1909639c11b925620ea, 91b2a010e1e45e6ba3d133fa997597be, ecab90c9933c58908d3d6add7c6f5ae3, f70a0aff17df5a6cdd9a7196128bd354, 1790eea0b567cf50911c057cf20f90f9, 989225ba6d0ebd5873335f7e01de2ae7, d887b52c6516beb39e8cd44a5f8b60f7, b276e4f8c0fb86bd82fce576f21713e0, 8563039e855156e48fccee4d611a3196, 60550084e6b4c0cb89a87df1f3e5ebd9, 5acce57f8d9dfd55fa48e212a641a69d, 434d158e96bdd6972ad6e6d73ddcfd22, 7206b86ea789983f7a273ea7fa0bc2a8, 1e7aff52cdbb2451ace09d0f848c3699, 6ea2f835b4556291ffdc53fa0b3b95e8, 948097deef559c742e7ce321e5e58919, d22e9fa5731b9e30e8b27afcdc2f8563, 6ebaec694d7025e2ad4a05dba887c032, d17dc4a904426827ca80f2ccb3a6be56, 25f4376934e13d3508486352e11a5db0, 5820a1100976432c7968a52da59e9364, 2ce1ad82022c1ba30c2079502ac725aa, 138849fd84dff2fb4ca70a0a34c4aa1c, 47aa4816b27ba60ec948cd019cc1afc1, 9faeb9b2746b9d7526aef5acb08e2aa0, 641fb0752bf5b5940c376b3a8bb9dc52, e425680f760cbc130be3e53a9773c584, 40c5e18f7d112b59b3e5113a59a905b3, 734e7d1bbaeb2ff82521ca0fe6fb6f79, 66e4624ae69e7dc89bd50222b59f581f, a685d016c8a26f71a0bb67821070e398, 2edfd6d1f0b4cd0db4bf37b1b224d855, 68873cf91053cd11e6b49a766db5af1a, f346ad4ee8f630e5e4ddaf862a34e6dd, 8f06cc6465925031568537b815f1198d, ccbabeb0b02433bd0fcbac46e70339f2, 688052146432ef8253587b930b01a06d, f271576bed568e896f99eb710cd3a6f8, 686541986ecfb7d9296eb67719973bf0, 68e48e68da1f50f7c5838ea75e3a20dd, b52cc4919de82b4d696a4380d10804a3, fdf128b3630c21adc9ca4fb8a51b68ec, a6aeb116d2cb5013eb8a94585b71ffef, fa516182d28f96f5f5c651026b0749ee, 6abaad69b8b349c3a529b4b91ce18e46, 974c1993ab8024d3ed16229183c2308d, 82bce245b1c9148f8d19a55b9ff70644, a910f58086d58b3ae6f37aa712d377b9, bd4bd0194d6d29f83b8557d4b89b572a, 634e8f4c0f6744a626f77f39770ac6aa, 6d25592267349b322799e2beb687871e, b8801cccd8068de30112e4f49903d74a, 2711a938db643b3f0b62ee2c8a2784aa, 3bc77ce8be27211bac313c2daa402d1a, 10c320f977c6a18f91b2d14be13128c6, 0a4a2fccb27bd83a892fa503987a595b, e4de6d53ecff736bc68804b0b6e9f635, 6b860b35691d486e45dc98e3514ec5f6, ec341c54a5ebf8ee0a67a8632aa7579b, cadbb3657dac2dbbd5b84b12e7b78aad, 9defaf92cff22420e4e8ef7784815a55, 20e0101b20700188cadb288126949685, 0e782c3705510e717d28907746cbda82, d3d6788577c9592da441752e8a1dd5e3, 86f21bf63784876b9fd6d35f46581d72, 8447ff843b2616c50c0ced28ab1dae03, f169bd689fb8b32ccd62df9050aebc0b, 77e9941864fc840be8e4b1ba5347c0f7, 41bb5cee06dbf170878a9ef93ac7e7f5, 6a0a8bfbbe700284feb0845d95e0867f, f7959f8385f34c4f645327465a1c9fc4, 23f553848a03aaab35bb3f9f87725125, e3447938231fb6ba2fee0231b51eca59, e346cd9299371b18c0b28e8e29a5e376, ...]"
1,purchase_to_approve_hrs,32616,"[0.17833333333333334, 30.71388888888889, 0.2761111111111111, 0.2980555555555556, 1.0305555555555554, 0.21888888888888888, 0.1947222222222222, 32.36055555555556, 0.175, 0.15222222222222223, 0.19916666666666666, 41.11472222222222, 0.18444444444444444, 0.9386111111111111, 124.46388888888889, 0.13916666666666666, 0.1475, 0.17805555555555555, 0.23, 13.544166666666667, 0.2808333333333333, 0.21305555555555555, 0.20055555555555554, 15.669722222222223, 26.73138888888889, 24.5825, 1.6355555555555557, 25.15638888